In [2]:
import pandas as pd
import numpy as np
import json

In [4]:
# Getting comments on posts where physicians have responded
physician_comments = []

# Read and filter comments for physician responses
with open("/Users/anikabansal/Documents/Junior Year/NLP/NLP-Final-project/AskDocs_comments.csv", "r") as f:
    for line in f:
        comment = json.loads(line)

        if comment.get("author_flair_text") == "Physician":
            physician_comments.append(comment)

df_physician_comments = pd.DataFrame(physician_comments)

# Clean up link_id field
if "link_id" in df_physician_comments.columns:
    df_physician_comments["clean_link_id"] = df_physician_comments["link_id"].str.replace("t3_", "", regex=False)

In [5]:
# Only keep highest scored physician comment
df_top_comments = df_physician_comments.loc[df_physician_comments.groupby("clean_link_id")["score"].idxmax()]
df_top_comments.head()

,author_flair_css_class,distinguished,downs,name,author_flair_text,retrieved_on,archived,score_hidden,score,controversiality,...,author_premium,treatment_tags,top_awarded_type,comment_type,collapsed_reason_code,retrieved_utc,author_is_blocked,unrepliable_reason,media_metadata,clean_link_id
126717,verified-doc,None,NaN,t1_j2ev7yt,Physician,1.673023e+09,False,False,1,0,...,False,[],NaN,NaN,None,NaN,NaN,NaN,NaN,1000e90
126719,verified-doc,None,NaN,t1_j2ew7lx,Physician,1.673023e+09,False,False,2,0,...,False,[],NaN,NaN,None,NaN,NaN,NaN,NaN,1001crw
126718,verified-doc,None,NaN,t1_j2ew1q4,Physician,1.673023e+09,False,False,3,0,...,True,[],NaN,NaN,None,NaN,NaN,NaN,NaN,1001f1r
126723,verified-doc,None,NaN,t1_j2f1h6c,Physician,1.673023e+09,False,False,9,0,...,False,[],NaN,NaN,None,NaN,NaN,NaN,NaN,1001sio
126727,verified-doc,None,NaN,t1_j2fgdk7,Physician,1.673022e+09,False,False,6,0,...,False,[],NaN,NaN,None,NaN,NaN,NaN,NaN,1004fot


In [6]:
df_top_comments.shape

(80870, 77)

In [7]:
# Get unique post IDs that have physician responses
physician_post_ids = set(df_physician_comments.groupby("clean_link_id")["score"].idxmax().index)

filtered_submissions = []
found_count = 0
max_posts = 1000

# Read and filter submissions
with open("/Users/anikabansal/Documents/Junior Year/NLP/NLP-Final-project/AskDocs_submissions.csv", "r") as f:
    for line in f:
        submission = json.loads(line)
        
        if submission["id"] in physician_post_ids:
            filtered_submissions.append(submission)
            found_count += 1

        if found_count >= max_posts:
            break

df_filtered_submissions = pd.DataFrame(filtered_submissions)

# Display first few rows
df_filtered_submissions.head()
print(f"Saved {found_count} submissions with physician responses.")


Saved 1000 submissions with physician responses.


In [8]:
merged_df = df_filtered_submissions.merge(
    df_top_comments,
    left_on="id", 
    right_on="clean_link_id", 
    how="left",
    suffixes=("_submission", "_comment")
)

merged_df.rename(columns={"body": "physician_comments"}, inplace=True)

# Save to CSV
merged_df.to_csv("merged_physician_submissions_1000.csv", index=False)

# Display first few rows
print(f"Saved {len(merged_df)} merged records.")
merged_df.head()

Saved 1000 merged records.


,subreddit_submission,created_utc_submission,report_reasons_submission,is_self,domain,permalink_submission,stickied_submission,title,gilded_submission,secure_media,...,author_premium,treatment_tags,top_awarded_type,comment_type,collapsed_reason_code,retrieved_utc,author_is_blocked,unrepliable_reason,media_metadata,clean_link_id
0,AskDocs,1373720814,NaN,True,self.AskDocs,/r/AskDocs/comments/1i7s0u/leg_pain_after_4ml_...,False,Leg pain after 4ml IM injection,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1i7s0u
1,AskDocs,1373718621,NaN,True,self.AskDocs,/r/AskDocs/comments/1i7qst/exhausted_out_of_br...,False,"Exhausted, out of breath",0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1i7qst
2,AskDocs,1373700194,NaN,True,self.AskDocs,/r/AskDocs/comments/1i7hzo/do_these_symptoms_m...,False,Do these symptoms mean I should get a colonosc...,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1i7hzo
3,AskDocs,1373843382,NaN,True,self.AskDocs,/r/AskDocs/comments/1iawxh/how_can_i_fight_my_...,False,How can I fight my Rosacea problem,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1iawxh
4,AskDocs,1373806677,NaN,True,self.AskDocs,/r/AskDocs/comments/1i9v7i/thought_i_was_going...,False,Thought I was going to die yesterday.,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1i9v7i


In [9]:
merged_df.shape

(1000, 121)